# **Behavioral Cloning** 
## Autonomous Mode Display

In [1]:
from IPython.display import HTML
HTML("""
<video width="600" height="200" controls>
  <source src="{0}">
</video>
""".format('run_best.mp4'))

## Data Collection

In [1]:
import csv
import cv2
import numpy as np

#lines = []
car_images, steering_angles = [], []
test = []

with open('data/driving_log.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None) # skip the column names
    
    for row in reader:
        steering_center = float(row[3])   

        # Create adjusted steering measurements for the side camera images
        correction = 0.2 # required tuning      
        steering_left = steering_center + correction
        steering_right = steering_center - correction      

        # Read in images from center, left, and right cameras
        path = 'data/'
        img_center = cv2.imread(path + row[0].strip())
        img_left = cv2.imread(path + row[1].strip()) 
        img_right = cv2.imread(path + row[2].strip())   

        # Add images and angles to the data set
        car_images.extend([img_center, img_left, img_right])
        test.append(img_center)
        test.append(img_left)
        steering_angles.extend([steering_center, steering_left, steering_right])

In [2]:
# Data augmentation
augmented_images, augmented_angles = [], []
for image, angle in zip(car_images, steering_angles):
    augmented_images.append(image)
    augmented_angles.append(angle)
    augmented_images.append(np.fliplr(image))
    augmented_angles.append(-angle)
    
X_samples = np.array(augmented_images)
y_samples = np.array(augmented_angles)

In [3]:
# Shuffle the data
from sklearn.model_selection import train_test_split
X_train_samples, X_valid_samples, y_train_samples, y_valid_samples = train_test_split(X_samples, y_samples, test_size=0.2, random_state=42)

## Model Design

In [4]:
import numpy as np
import sklearn

batch_size = 48

def generator(X_samples, y_samples, batch_size=batch_size):
    num_samples = len(y_samples)
    while 1: # Loop forever so the generator never terminates
        sklearn.utils.shuffle(X_samples, y_samples)
        for offset in range(0, num_samples, batch_size):
            X_batch = np.array(X_samples[offset:offset+batch_size])
            y_batch = np.array(y_samples[offset:offset+batch_size])
            yield (X_batch, y_batch)

# Load the trained model            
#from keras.models import load_model
#model = load_model('model_best.h5')

from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Lambda, Convolution2D, Cropping2D, Dropout
from keras.layers.pooling import MaxPooling2D

model = Sequential() # create a new model
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160, 320, 3)))
model.add(Cropping2D(cropping=((70, 25), (0, 0)), input_shape=(3, 160, 320)))
model.add(Convolution2D(24, 5, 5, subsample=(2, 2), activation='relu'))
model.add(Convolution2D(36, 5, 5, subsample=(2, 2), activation='relu'))
model.add(Convolution2D(48, 5, 5, activation='relu')) # adjustment made
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu')) # adjustment made
model.add(MaxPooling2D()) # adjustment made
model.add(Flatten())
model.add(Dense(100))
model.add(Dropout(0.5)) # adjustment made
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

# Compile and train the model using the generator function
train_generator = generator(X_train_samples, y_train_samples, batch_size=batch_size)
validation_generator = generator(X_valid_samples, y_valid_samples, batch_size=batch_size)

model.compile(loss='mse', optimizer='adam')
history_object = model.fit_generator(train_generator, samples_per_epoch=len(X_train_samples),
                    validation_data=validation_generator, nb_val_samples=len(X_valid_samples),
                    nb_epoch=3, verbose=1)

import matplotlib.pyplot as plt

### Print the keys contained in the history object
print(history_object.history.keys())

### Plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()

Using TensorFlow backend.


Epoch 1/3
38572/38572 [==============================] - 925s - loss: 0.0200 - val_loss: 0.0173
Epoch 2/3
38572/38572 [==============================] - 816s - loss: 0.0166 - val_loss: 0.0160
Epoch 3/3
38572/38572 [==============================] - 808s - loss: 0.0157 - val_loss: 0.0151
dict_keys(['val_loss', 'loss'])


In [6]:
# The very first model output without any adjustment to the original setting
#model.save('model.h5') # batch size =32
#exit()

In [ ]:
model.save('model_best.h5') # model updated with new data
exit()

## Flip Image

In [14]:
# flip imager
import cv2
import numpy as np
path = 'examples/'
image = cv2.imread(path + 'center.jpg')
image_flip = np.fliplr(image)
cv2.imwrite(path + 'center_flip.jpg', image_flip)

True

## Video Output

Execute the commands below in the terminal:

<font color='blue'>python drive.py model_best.h5 run_best</font><br/>

<font color='blue'>python video.py run_best --fps 48</font>